<h1> Information Security, Assignment 5 - Bonus Question </h1>
<h1> Pytharoulios Konstantinos (Pythak@usi.ch) </h1> 
<h1> Spanopoulos Andreas - Theologos (spanoa@usi.ch) <h1>

Start by including the main S-AES library and also defining important sizes for the CMAC.

In [1]:
# import the built Mini-AES library
from sage.crypto.block_cipher.miniaes import MiniAES
crypto = MiniAES()
bin = BinaryStrings()

# the sizes indicate number of bits. Both should be a multiple of 8.
# Everything in the code is generic (depending on these size variables), but the MiniAES() works with 16 bits
# so these sizes should not be changed
block_size = 16
key_size = 16

Define the plaintext that we want to encode and the key to be used for it.

In [2]:
# change here the plaintext that you want to encrypt
plaintext = 'I should have started using Github way long ago'
# change here the key that you want to use for encryption
# the key values can range from 0 up to 2^{block_size} - 1 = 2^{16} - 1 = 65535
K = 420

Now we have to split the plaintext in blocks of length (block_size)-bits. Note that each ASCII symbol is 8 bits long.

In [3]:
# how many bits the plaintext has
plaintext_bits = len(plaintext) * 8
# number of messages
n = plaintext_bits // block_size

# check if the last message will need padding
needs_padding = False
bits_remaining = plaintext_bits % block_size
if bits_remaining != 0:
    needs_padding = True
    n = n + 1
    
# how many bytes each block will contain
bytes_per_block = block_size // 8

# create a list which will have blocks of plaintext messages
list_of_block_messages = []
for i in range(n - 1):
    temp_list = []
    for byte in range(0, bytes_per_block):
        temp_list.append(plaintext[bytes_per_block * i + byte])
    
    list_of_block_messages.append(''.join(temp_list))
    
    
# calculate how many bytes the last message has. If the message fits exactly, then it has bytes_per_block bytes
bytes_left = bytes_per_block if (bits_remaining == 0) else (bits_remaining // 8)
# the last message has to be handled explicitly because it may need padding
temp_list = []
for byte in range(bytes_left):
    temp_list.append(plaintext[bytes_per_block * (n - 1) + byte])
    
# if needed, pad 1 followed by 0's until the block size is reached
if needs_padding:
    # how many bytes need to be padded
    bytes_to_be_padded = bytes_per_block - bytes_left
    # pad the first byte: 10000000
    first_padded_byte = chr(int('0b10000000', 2))
    temp_list.append(first_padded_byte)
    bytes_to_be_padded = bytes_to_be_padded - 1
    
    # if there are more bytes to be padded, do it
    while bytes_to_be_padded > 0:
        # pad the next byte: 00000000
        temp_list.append(chr(int('0b00000000', 2)))
        bytes_to_be_padded = bytes_to_be_padded - 1
    
# add the last message block to the list
list_of_block_messages.append(''.join(temp_list))

Function that converts binary strings to their respective ASCII symbols string.

In [4]:
# converts a binary string to it's respective Ascii form. For example: "01000001" -> "A"
# or "010000010100001001000011" -> "ABC"
def Binary_to_Ascii_string(string):
    ascii_list = []
    bytes_in_string = len(string) // 8
    for i in range(bytes_in_string):
        byte = int(string[i * 8 : i * 8 + 8], 2)
        ascii_list.append(chr(byte))
    
    return ''.join(ascii_list)

Now we have to calculate the keys $K_1$ and $K_2$.

In [5]:
# how many bytes the key has
bytes_in_key = key_size // 8

# convert the key to a 16-bit binary string
key_as_binary_string = '{0:016b}'.format(K)
key_in_ascii = Binary_to_Ascii_string(key_as_binary_string)
key = bin.encoding(key_in_ascii)
    
# create a (key_size)-bit string of zeros
zeros = []
for i in range(bytes_in_key):
    zeros.append("\0")

# the string is ready, but we have to use the encoding function
string_of_zeros = bin.encoding(''.join(zeros))

# compute L
L_encoding = crypto(string_of_zeros, key, algorithm = "encrypt");
L = str(L_encoding)

# create the Galois Field
F.<x> = GF(2^block_size)

# compute F(L) so that we stay inside the field

# first we need to represent L in a reversed bit-list
L_as_list = []
for i in range(len(L)):
    L_as_list.append(L[i])
    
# reverse the list
L_as_list.reverse()

# now we can reduce it
reduced_L = F(L_as_list)


# compute K_1
K_1 = reduced_L * x
K_1_as_list = K_1.polynomial().coefficients(sparse = False)

# fill with 0's if needed
zeros_needed = key_size - len(K_1_as_list)
for i in range(zeros_needed):
    K_1_as_list.append(0)
    
# reverse it because the coefficients are in the opposite order
K_1_as_list.reverse()


# compute K_2
K_2 = K_1 * x
K_2_as_list = K_2.polynomial().coefficients(sparse = False)

# fill with 0's if needed
zeros_needed = key_size - len(K_2_as_list)
for i in range(zeros_needed):
    K_2_as_list.append(0)
    
# reverse it because the coefficients are in the opposite order
K_2_as_list.reverse()

Useful conversion functions.

In [6]:
# converts list of binary chars to ints. For example: ['0', '1', '0'] -> [0, 1, 0]
def Char_to_Int_List(string):
    bin_list = []
    for ch in string:
        if ch == '0':
            bin_list.append(0)
        else:
            bin_list.append(1)
        
        
    return bin_list

# does the opposite of the above function
def Int_to_Char_List(string):
    char_list = []
    for num in string:
        if num == 0:
            char_list.append('0')
        else:
            char_list.append('1')
            
    return char_list

Function used to calculate the exclusive-or (xor) of two strings, stolen from the labs :).

In [7]:
def XorBlock(block1, block2):
    l = len(block1);
    if (l != len(block2)):
        raise (ValueError, "XorBlock arguments must be same length")
    return [(block1[j] + block2[j]) % 2 for j in range(l)];

Now we have to calculate the ciphertexts.

In [8]:
# encode the first plaintext in the format needed
plaintext_encoding = bin.encoding(list_of_block_messages[0])
# compute c_1
c1 = crypto(plaintext_encoding, key, algorithm = 'encrypt')

# make a list of ciphertexts in order to print them in the end
ciphertexts = [c1]
# store the previous ciphertext in order to "xor" it with the next plaintext
previous_ciphertext = c1

# for the block ciphertexts 2 : n
for i in range(2, n + 1):
    
    # calculate the encoding of the next message
    plaintext_encoding = bin.encoding(list_of_block_messages[i - 1])
    
    # translate it as a list of of 0's and 1's in order to be able to do the xor operation
    plaintext_encoding_as_list = Char_to_Int_List(list(str(plaintext_encoding)))

    # same for the previous ciphertext
    previous_ciphertext_as_list = Char_to_Int_List(list(str(previous_ciphertext)))
    
    # xor the plaintext with the previous ciphertext
    xored_string_as_list = XorBlock(plaintext_encoding_as_list, previous_ciphertext_as_list)
    
    # if we are computing the last ciphertext C_n, then we have to also xor with one of the keys k1 or k2,
    # depending on if we padded or not
    if i == n:
        if needs_padding:
            xored_string_as_list = XorBlock(xored_string_as_list, K_2_as_list)
        else:
            xored_string_as_list = XorBlock(xored_string_as_list, K_1_as_list)
            
    # get the xored plaintext as a string of 0' and 1's
    xored_string = ''.join(Int_to_Char_List(xored_string_as_list))
    # convert the string to Ascii format in order to use bin.encoding()
    xored_string_as_ascii = Binary_to_Ascii_string(xored_string)
    
    # finally encode the xored string in the format needed
    xored_string_encoded = bin.encoding(xored_string_as_ascii)
   
    # encrypt the thing
    c_i = crypto(xored_string_encoded, key, algorithm = 'encrypt')
    # store the new ciphertext
    previous_ciphertext = c_i
    # add it to the list
    ciphertexts.append(c_i)
    
    
# compute T
C_n = ciphertexts[n - 1]
C_n_as_list = Char_to_Int_List(list(str(C_n)))

# get the leftmost bits
T_as_list = C_n_as_list
# if we had padding
if needs_padding != 0:
    # take those that we need
    T_as_list = C_n_as_list[0 : bits_remaining]
    
# get T as a string of bits
T = ''.join(Int_to_Char_List(T_as_list))

Now we can print everything to see the results:

In [9]:
print('There are totally', n, 'plaintext block messages to be encrypted:\n')
for i in range(n):
    print('C_{', i + 1, '} = ', ciphertexts[i])
print('T = ', T)

There are totally 24 plaintext block messages to be encrypted:

C_{ 1 } =  0000100110100101
C_{ 2 } =  0011001101110001
C_{ 3 } =  0100000001010110
C_{ 4 } =  1001110100111001
C_{ 5 } =  1110111110100111
C_{ 6 } =  1000100001110000
C_{ 7 } =  1100111110101110
C_{ 8 } =  0110010001100100
C_{ 9 } =  1000101010100100
C_{ 10 } =  1100011110000110
C_{ 11 } =  1010010011111101
C_{ 12 } =  1100101101010100
C_{ 13 } =  1110000011000110
C_{ 14 } =  0011100010100101
C_{ 15 } =  0100110000001110
C_{ 16 } =  1110001011111110
C_{ 17 } =  0111011110110101
C_{ 18 } =  0110010001011111
C_{ 19 } =  1000011001000100
C_{ 20 } =  0101101010111111
C_{ 21 } =  1101110011000011
C_{ 22 } =  1100111001100011
C_{ 23 } =  1111011101101011
C_{ 24 } =  0000111100010101
T =  00001111
